In [1]:
import numpy as np

In [2]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None   # link similar items (the dashed lines in figure 12.1)
        self.parent = parentNode
        self.children = {}     # an empty dictionary for the children of this node
        
    #  increments the  count variable by a given amount
    def inc(self, numOccur):
        self.count += numOccur
    
    # display the tree in text, for debugging
    def disp(self, ind=1):
        print (' '*ind, self.name, ' ', self.count)  # 左对齐显示
        for child in self.children.values():
            child.disp(ind+1)

In [4]:
rootNode = treeNode('pyramid',9, None)
rootNode.children['eye']=treeNode('eye', 13, None)
rootNode.disp()

  pyramid   9
   eye   13


In [5]:
rootNode.children['phoenix']=treeNode('phoenix', 3, None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


In [142]:
def createTree(dataSet, minSup=1):
    itemCount, headerTable = {}, {}
    # The first pass goes through everything in the dataset and counts the frequency of each term. 
    for transaction in dataSet:
        for item in transaction:
            itemCount[item] = itemCount.get(item, 0) + 1# dataSet[trans]
    print ('item count = ', itemCount)
    # the header table is scanned and items occurring less than  minSup are deleted.
    for key in itemCount.keys():
        if itemCount[key] >= minSup:
            headerTable[key] = [itemCount[key], None] # he header table hold a count and pointer to the first item of each type.
    print ('headerTable = ',headerTable)
    # create the base node, which contains the null set Ø
    rootNode = treeNode('Null Set', 1, None)
    # iterate over the dataset again
    for transaction in dataSet:
        # 把frequent低的item过滤掉
        localD = []
        for item in transaction:
            if item in headerTable: # 在headerTable中的都是Frequent达到要求的
                localD.append(item)
        # 过滤之后排序  
        localD.sort(key=lambda k:(headerTable[k][0]), reverse=True)
        #print ('localD = ', localD)
        # 将得到的新的Transaction插入到tree中
        node = rootNode
       
        for item in localD:
            node = updateTree(item, node, headerTable, 1)
        #print ('tree after one transaction', rootNode.disp())
    return rootNode, headerTable

# 把item插入到以node为父结点的树中
def updateTree(item, node, headerTable, count):
    if item in node.children:
        node.children[item].count+=1
    else:
        node.children[item] = treeNode(item, count, node)
        if headerTable[item][1] == None:
            headerTable[item][1] = node.children[item]
        else:
            updateHeader(headerTable[item][1], node.children[item])
    return node.children[item]

# headerTable和后面结点的nodeLink组成一个单链表
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [143]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
    ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
    ['z'],
    ['r', 'x', 'n', 'o', 's'],
    ['y', 'r', 'x', 'z', 'q', 't', 'p'],
    ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [144]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [145]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [146]:
initSet = createInitSet(simpDat) # 为什么要转成frozenset？
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [147]:
myFPtree, myHeaderTab = createTree(initSet, 3)

item count =  {'z': 5, 'j': 1, 'r': 3, 'p': 2, 'h': 1, 'u': 1, 'x': 4, 'v': 1, 't': 3, 'y': 3, 'w': 1, 's': 3, 'o': 1, 'n': 1, 'q': 2, 'm': 1, 'e': 1}
headerTable =  {'z': [5, None], 'r': [3, None], 'x': [4, None], 't': [3, None], 'y': [3, None], 's': [3, None]}


In [148]:
myFPtree.disp() # 对相同Frequent的item，排序结果与书上不同，导致树与书上不同

  Null Set   1
   z   5
    r   1
    x   3
     t   3
      y   3
       s   2
       r   1
   x   1
    r   1
     s   1


In [110]:
myHeaderTab

{'z': [5, <__main__.treeNode at 0x171a1f044c8>],
 'r': [3, <__main__.treeNode at 0x171a1f04388>],
 'x': [4, <__main__.treeNode at 0x171a24e92c8>],
 't': [3, <__main__.treeNode at 0x171a1c231c8>],
 'y': [3, <__main__.treeNode at 0x171a1c23488>],
 's': [3, <__main__.treeNode at 0x171a1c23d48>]}

In [153]:
def ascendTree(treeNode):  # 这个node不一定是leaf
    prefix = []
    while(treeNode.parent != None):
        prefix.append(treeNode.name)
        treeNode = treeNode.parent
    return prefix

# basePath: 暂时没用上
# treeNode: headTable[item]
def findPrefixPath(basePath, treeNode):
    prefixPaths = {}
    while(treeNode != None):
        prefix = ascendTree(treeNode)
        if len(prefix) > 1:
            prefixPaths[frozenset(prefix[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return prefixPaths

In [154]:
# myHeaderTab是个set
# set的key是一个item，set的value是个list
# list的第0个元素是这个item的count，第1个元素是这个item的第一个node的link
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [155]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [156]:
findPrefixPath('r', myHeaderTab['r'][1]) #生成的树不同，导致path不同

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'t', 'x', 'y', 'z'}): 1}

In [163]:
# freqItemList：所有满足minSup的Set都存到这里
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # sorting the items in the header table by their frequency of occurrence
    sortedKey = [item[0] for item in sorted(headerTable.items(),key=lambda p: p[0])]
    # Construct cond. FP-tree from cond. pattern base
    for key in sortedKey:
        # preFix和{preFix,key}都是满足minSup的set，应该都存在于freqItemList的中，因为要copy
        # 如果不copy，freqItemList中只有{preFix,key}，而preFix就丢失了
        newFreqSet = preFix.copy()
        newFreqSet.add(key)
        # each frequent item is added to your list of frequent itemsets
        freqItemList.append(newFreqSet)
        
        # 新的一轮迭代
        # create a conditional base
        condPattBases = findPrefixPath(key, headerTable[key][1])
        print(key, condPattBases)
        # This conditional base is treated as a new dataset and fed to  createTree() 
        myCondTree, myHead = createTree(condPattBases,minSup)
        # Mine cond. FP-tree
        if myHead != None:
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [ ]:
# createTree假设所有Data出现即frequency+1
# 在现在的应该场景中，dataSet的每一项是个dict，dict的Value说明dict的key中的data出现一次代表frequency+几
def createTree_2(dataSet, minSup=1):
    itemCount, headerTable = {}, {}
    # The first pass goes through everything in the dataset and counts the frequency of each term. 
    for transaction in dataSet:
        for item in transaction:
            itemCount[item] = itemCount.get(item, 0) + 1# dataSet[trans]
    print ('item count = ', itemCount)
    # the header table is scanned and items occurring less than  minSup are deleted.
    for key in itemCount.keys():
        if itemCount[key] >= minSup:
            headerTable[key] = [itemCount[key], None] # he header table hold a count and pointer to the first item of each type.
    print ('headerTable = ',headerTable)
    # create the base node, which contains the null set Ø
    rootNode = treeNode('Null Set', 1, None)
    # iterate over the dataset again
    for transaction in dataSet:
        # 把frequent低的item过滤掉
        localD = []
        for item in transaction:
            if item in headerTable: # 在headerTable中的都是Frequent达到要求的
                localD.append(item)
        # 过滤之后排序  
        localD.sort(key=lambda k:(headerTable[k][0]), reverse=True)
        #print ('localD = ', localD)
        # 将得到的新的Transaction插入到tree中
        node = rootNode
       
        for item in localD:
            node = updateTree(item, node, headerTable, 1)
        #print ('tree after one transaction', rootNode.disp())
    return rootNode, headerTable

# 把item插入到以node为父结点的树中
def updateTree(item, node, headerTable, count):
    if item in node.children:
        node.children[item].count+=1
    else:
        node.children[item] = treeNode(item, count, node)
        if headerTable[item][1] == None:
            headerTable[item][1] = node.children[item]
        else:
            updateHeader(headerTable[item][1], node.children[item])
    return node.children[item]

# headerTable和后面结点的nodeLink组成一个单链表
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [164]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

r {frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'y', 'z', 't', 'x'}): 1}
item count =  {'z': 2, 'x': 2, 'y': 1, 't': 1}
headerTable =  {}
s {frozenset({'y', 'z', 't', 'x'}): 2, frozenset({'x', 'r'}): 1}
item count =  {'y': 1, 'z': 1, 't': 1, 'x': 2, 'r': 1}
headerTable =  {}
t {frozenset({'z', 'x'}): 3}
item count =  {'z': 1, 'x': 1}
headerTable =  {}
x {frozenset({'z'}): 3}
item count =  {'z': 1}
headerTable =  {}
y {frozenset({'z', 't', 'x'}): 3}
item count =  {'z': 1, 't': 1, 'x': 1}
headerTable =  {}
z {}
item count =  {}
headerTable =  {}
